# Analisi Back Squat con Accelerometro

Questo notebook analizza i dati dell'accelerometro durante l'esecuzione di back squat, identificando automaticamente le diverse fasi del movimento:
- **Posizione iniziale (Standing)**
- **Fase eccentrica (Discesa)**
- **Posizione inferiore (Bottom)**
- **Fase concentrica (Salita)**

I dati vengono visualizzati con marker che indicano i cambiamenti di fase.

## 1. Import Librerie

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

# Configurazione per grafici più grandi e leggibili
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

print("✅ Librerie importate con successo")

## 2. Caricamento Dati

In [1]:
# Carica il file CSV
filename = ''

df = pd.read_csv(filename)

# Mostra le prime righe
print(f"📊 Dataset caricato: {len(df)} campioni")
print(f"⏱️  Durata: {df['Timestamp'].max() - df['Timestamp'].min():.2f} secondi")
print(f"📈 Frequenza di campionamento: ~{len(df)/(df['Timestamp'].max() - df['Timestamp'].min()):.1f} Hz\n")

df.head(10)

NameError: name 'pd' is not defined

## 3. Visualizzazione Dati Grezzi

In [ ]:
# Plot dei dati grezzi
fig, axes = plt.subplots(4, 1, figsize=(14, 10), sharex=True)

axes[0].plot(df['Timestamp'], df['X (g)'], 'r-', alpha=0.7, linewidth=1)
axes[0].set_ylabel('X (g)', fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].set_title('Accelerazione sui 3 assi + Magnitudine', fontsize=14, fontweight='bold')

axes[1].plot(df['Timestamp'], df['Y (g)'], 'g-', alpha=0.7, linewidth=1)
axes[1].set_ylabel('Y (g)', fontweight='bold')
axes[1].grid(True, alpha=0.3)

axes[2].plot(df['Timestamp'], df['Z (g)'], 'b-', alpha=0.7, linewidth=1)
axes[2].set_ylabel('Z (g)', fontweight='bold')
axes[2].grid(True, alpha=0.3)

axes[3].plot(df['Timestamp'], df['Magnitude (g)'], 'purple', linewidth=2)
axes[3].set_ylabel('Magnitudine (g)', fontweight='bold')
axes[3].set_xlabel('Tempo (s)', fontweight='bold')
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Smoothing e Preprocessing

In [ ]:
# Applica smoothing gaussiano per ridurre il rumore
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

sigma = 2  # Parametro di smoothing

df['X_smooth'] = gaussian_filter1d(df['X (g)'], sigma=sigma)
df['Y_smooth'] = gaussian_filter1d(df['Y (g)'], sigma=sigma)
df['Z_smooth'] = gaussian_filter1d(df['Z (g)'], sigma=sigma)
df['Mag_smooth'] = gaussian_filter1d(df['Magnitude (g)'], sigma=sigma)

# Calcola la derivata (velocità) della magnitudine per identificare cambiamenti
df['Mag_velocity'] = np.gradient(df['Mag_smooth'], df['Timestamp'])
df['Mag_velocity_smooth'] = gaussian_filter1d(df['Mag_velocity'], sigma=sigma)

print("✅ Smoothing applicato")

## 5. Rilevamento Fasi dello Squat

L'algoritmo identifica l'**inizio** di ogni fase del back squat:
1. **Baseline** - Posizione iniziale in piedi (stabile)
2. **Eccentrica** - Inizio discesa (accelerazione negativa su Y)
3. **Concentrica** - Inizio spinta verso l'alto (punto più basso + cambio direzione)
4. **Arresto** - Fine movimento (decelerazione vicino al top)
5. **Baseline** - Ritorno alla stabilità

In [ ]:
# Analisi Magnitude con Soglie intorno alla Baseline
# Definiamo due soglie per rilevare quando la baseline viene rotta

mag = df['Mag_smooth'].values
timestamps = df['Timestamp'].values

print("🔍 ANALISI MAGNITUDE CON SOGLIE BASELINE")
print("=" * 80)

# Calcola baseline dai primi campioni stabili (primi 20%)
baseline_samples = int(len(mag) * 0.2)
baseline_value = np.median(mag[:baseline_samples])

# Definiamo due soglie simmetriche intorno alla baseline
threshold_percent = 0.05  # 5% della baseline
upper_threshold = baseline_value + (baseline_value * threshold_percent)
lower_threshold = baseline_value - (baseline_value * threshold_percent)

print(f"📊 Baseline: {baseline_value:.3f}g")
print(f"📊 Soglia superiore (+5%): {upper_threshold:.3f}g")
print(f"📊 Soglia inferiore (-5%): {lower_threshold:.3f}g")

# Trova quando le soglie vengono rotte
upper_breaks = np.where(mag > upper_threshold)[0]
lower_breaks = np.where(mag < lower_threshold)[0]

print(f"\n🔴 Soglia superiore rotta in {len(upper_breaks)} punti")
if len(upper_breaks) > 0:
    print(f"   Primo break: t={timestamps[upper_breaks[0]]:.2f}s (Mag={mag[upper_breaks[0]]:.3f}g)")
    print(f"   Ultimo break: t={timestamps[upper_breaks[-1]]:.2f}s (Mag={mag[upper_breaks[-1]]:.3f}g)")

print(f"\n🟢 Soglia inferiore rotta in {len(lower_breaks)} punti")
if len(lower_breaks) > 0:
    print(f"   Primo break: t={timestamps[lower_breaks[0]]:.2f}s (Mag={mag[lower_breaks[0]]:.3f}g)")
    print(f"   Ultimo break: t={timestamps[lower_breaks[-1]]:.2f}s (Mag={mag[lower_breaks[-1]]:.3f}g)")

print("=" * 80)

## 6. Visualizzazione con Marker delle Fasi

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

# Plot magnitudine
ax.plot(df['Timestamp'], df['Magnitude (g)'], 'lightgray', alpha=0.5, linewidth=1, label='Raw')
ax.plot(df['Timestamp'], df['Mag_smooth'], 'purple', linewidth=2.5, label='Smoothed')

# Linee delle soglie
ax.axhline(y=baseline_value, color='blue', linestyle='--', linewidth=2, label=f'Baseline ({baseline_value:.3f}g)', alpha=0.7)
ax.axhline(y=upper_threshold, color='red', linestyle='--', linewidth=1.5, label=f'Soglia Superiore (+5%)', alpha=0.7)
ax.axhline(y=lower_threshold, color='green', linestyle='--', linewidth=1.5, label=f'Soglia Inferiore (-5%)', alpha=0.7)

# Zona baseline (fill between)
ax.fill_between(df['Timestamp'], lower_threshold, upper_threshold, color='yellow', alpha=0.15, label='Zona Baseline')

# Evidenzia i punti dove le soglie vengono rotte
if len(upper_breaks) > 0:
    ax.scatter(timestamps[upper_breaks], mag[upper_breaks], color='red', s=10, alpha=0.3, label='Break Superiore')

if len(lower_breaks) > 0:
    ax.scatter(timestamps[lower_breaks], mag[lower_breaks], color='green', s=10, alpha=0.3, label='Break Inferiore')

ax.set_xlabel('Tempo (s)', fontweight='bold', fontsize=12)
ax.set_ylabel('Magnitudine (g)', fontweight='bold', fontsize=12)
ax.set_title('Analisi Magnitude con Soglie Baseline', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend(loc='upper right', fontsize=10, ncol=2)

plt.tight_layout()
plt.show()

## 7. Rilevamento Ripetizioni con Pattern Matching

Algoritmo basato su **sequenze di picchi di accelerazione**:

**FASE ECCENTRICA (discesa)**:
- Pattern: PICCO_GIÙ → BASE → (eventuale PICCO_SU per frenata)

**FASE CONCENTRICA (spinta)** ← qui calcoliamo VBT:
- Pattern: PICCO_SU → BASE → PICCO_GIÙ → BASE finale

L'algoritmo cerca queste sequenze caratteristiche invece di soglie assolute.

In [ ]:
"""
PATTERN MATCHING con VBT (Velocity Based Training)
Rilevamento ripetizioni tramite transizioni di stato + calcolo metriche VBT avanzate
"""

# Configura parametri
BASELINE_ZONE = 0.08  # ±8% variazione = stato BASE
MIN_REP_DURATION = 0.5  # secondi
MAX_REP_DURATION = 4.0  # secondi
REFRACTORY_PERIOD = 0.8  # secondi tra ripetizioni
PEAK_PROMINENCE = 0.15  # Prominenza minima dei picchi

# Calcola baseline
baseline_samples = 30
baseline_value = np.median(mag[:baseline_samples])
baseline_upper = baseline_value * (1 + BASELINE_ZONE)
baseline_lower = baseline_value * (1 - BASELINE_ZONE)

print(f"📊 BASELINE ANALYSIS")
print(f"   Baseline: {baseline_value:.3f}g")
print(f"   Upper threshold (+{BASELINE_ZONE*100:.0f}%): {baseline_upper:.3f}g")
print(f"   Lower threshold (-{BASELINE_ZONE*100:.0f}%): {baseline_lower:.3f}g")
print("=" * 90)

# Classifica samples in stati
signal_state = np.where(mag > baseline_upper, 1,  # ABOVE
                        np.where(mag < baseline_lower, -1,  # BELOW
                                 0))  # BASE

# Trova transizioni di stato
state_changes = []
for i in range(1, len(signal_state)):
    if signal_state[i] != signal_state[i-1]:
        state_changes.append({
            'idx': i,
            'time': timestamps[i],
            'from_state': signal_state[i-1],
            'to_state': signal_state[i],
            'mag': mag[i]
        })

print(f"🔄 State transitions detected: {len(state_changes)}")

# Pattern Matching: cerca BASE → movimento → BASE
valid_reps = []
last_rep_end_time = -REFRACTORY_PERIOD

for i in range(len(state_changes) - 1):
    if state_changes[i]['from_state'] == 0:  # Esce da BASE
        rep_start_idx = state_changes[i]['idx']
        rep_start_time = state_changes[i]['time']
        
        # Applica refractory period
        if rep_start_time - last_rep_end_time < REFRACTORY_PERIOD:
            continue
        
        # Cerca ritorno a BASE
        for j in range(i + 1, len(state_changes)):
            if state_changes[j]['to_state'] == 0:  # Ritorna a BASE
                rep_end_idx = state_changes[j]['idx']
                rep_end_time = state_changes[j]['time']
                rep_duration = rep_end_time - rep_start_time
                
                # Valida durata
                if MIN_REP_DURATION <= rep_duration <= MAX_REP_DURATION:
                    # Trova bottom (minimo) e picco concentrico
                    mag_segment = mag[rep_start_idx:rep_end_idx+1]
                    
                    if len(mag_segment) < 20:  # Minimo 20 samples
                        continue
                    
                    bottom_relative = np.argmin(mag_segment)
                    bottom_idx = rep_start_idx + bottom_relative
                    
                    # Verifica profondità (deve scendere sotto 0.90g)
                    if mag[bottom_idx] >= 0.90:
                        continue
                    
                    # Cerca picco concentrico dopo bottom
                    if bottom_relative < len(mag_segment) - 1:
                        concentric_segment = mag_segment[bottom_relative:]
                        concentric_peak_relative = np.argmax(concentric_segment)
                        concentric_peak_idx = bottom_idx + concentric_peak_relative
                    else:
                        concentric_peak_idx = rep_end_idx
                    
                    # Salva ripetizione valida
                    rep = {
                        'rep_num': len(valid_reps) + 1,
                        'start_idx': rep_start_idx,
                        'end_idx': rep_end_idx,
                        'bottom_idx': bottom_idx,
                        'concentric_peak_idx': concentric_peak_idx,
                        'start_time': rep_start_time,
                        'end_time': rep_end_time,
                        'bottom_time': timestamps[bottom_idx],
                        'concentric_peak_time': timestamps[concentric_peak_idx],
                        'duration': rep_duration,
                        'mag_start': mag[rep_start_idx],
                        'mag_bottom': mag[bottom_idx],
                        'mag_peak': mag[concentric_peak_idx],
                        'mag_end': mag[rep_end_idx]
                    }
                    valid_reps.append(rep)
                    last_rep_end_time = rep_end_time
                    break

print(f"\n✅ Valid repetitions detected: {len(valid_reps)}")
print("=" * 90)

# CALCOLO METRICHE VBT per ogni ripetizione
reps_with_metrics = []  # Solo le reps con metriche VBT valide

if len(valid_reps) > 0:
    for rep in valid_reps:
        # Estrai fase concentrica (bottom → peak)
        bottom_idx = rep['bottom_idx']
        concentric_peak_idx = rep['concentric_peak_idx']
        
        if concentric_peak_idx <= bottom_idx:
            print(f"⚠️  Rep {rep['rep_num']}: Invalid concentric phase (peak before bottom)")
            continue
        
        mag_concentric = mag[bottom_idx:concentric_peak_idx+1]
        time_concentric = timestamps[bottom_idx:concentric_peak_idx+1]
        
        if len(mag_concentric) < 2:
            print(f"⚠️  Rep {rep['rep_num']}: Too few samples in concentric phase")
            continue
        
        # GRAVITY COMPENSATION - Come tutti i dispositivi VBT seri (Vitruve, Beast, Enode, Metric VBT)
        # Sottrai SEMPRE 1.0g (non baseline variabile!) per evitare drift
        mag_accel_net = (mag_concentric - 1.0) * 9.81  # m/s²
        
        print(f"\n🔍 DEBUG Rep {rep['rep_num']}:")
        print(f"   Mag range: {mag_concentric.min():.3f}g to {mag_concentric.max():.3f}g")
        print(f"   Accel net range: {mag_accel_net.min():.3f} to {mag_accel_net.max():.3f} m/s²")
        
        # Integrazione velocità: v(t) = v(t-1) + a(t) * dt, con v(0) = 0 al bottom
        velocity = np.zeros(len(mag_accel_net))
        displacement = np.zeros(len(mag_accel_net))
        for k in range(1, len(mag_accel_net)):
            dt = time_concentric[k] - time_concentric[k-1]
            velocity[k] = velocity[k-1] + mag_accel_net[k] * dt
            displacement[k] = displacement[k-1] + velocity[k] * dt
        
        print(f"   Velocity range: {velocity.min():.3f} to {velocity.max():.3f} m/s")
        
        # Calcola propulsive mask PRIMA (per usarla dopo)
        propulsive_mask = mag_accel_net > 0
        
        # Metriche VBT di base
        positive_velocity = velocity[velocity > 0]
        if len(positive_velocity) > 0:
            mean_velocity = np.mean(positive_velocity)
            peak_velocity = np.max(velocity)
            # Mean Propulsive Velocity (MPV) - solo dove accelerazione è positiva
            if np.any(propulsive_mask):
                mean_propulsive_velocity = np.mean(velocity[propulsive_mask])
            else:
                mean_propulsive_velocity = 0.0
        else:
            mean_velocity = 0.0
            peak_velocity = 0.0
            mean_propulsive_velocity = 0.0
        
        # Time to Peak Velocity
        peak_vel_idx = np.argmax(velocity)
        time_to_peak_velocity = time_concentric[peak_vel_idx] - time_concentric[0]
        
        # ROM (Range of Motion) - displacement totale
        concentric_displacement = displacement[-1]  # Spostamento alla fine della fase concentrica
        
        # Calcolo potenza (P = m * a * v)
        # NOTA: velocity e mag_accel_net hanno stessa lunghezza ora
        MASS = 1.0  # kg (placeholder - da sostituire con massa reale)
        power = MASS * mag_accel_net * velocity  # Watt
        mean_power = np.mean(power[power > 0]) if np.any(power > 0) else 0.0
        peak_power = np.max(power)
        
        # Mean Propulsive Power - usa la stessa maschera propulsiva
        if np.any(propulsive_mask):
            mean_propulsive_power = np.mean(power[propulsive_mask])
        else:
            mean_propulsive_power = 0.0
        
        # Salva tutte le metriche
        rep['mean_velocity'] = mean_velocity
        rep['peak_velocity'] = peak_velocity
        rep['mean_propulsive_velocity'] = mean_propulsive_velocity
        rep['time_to_peak_velocity'] = time_to_peak_velocity
        rep['concentric_displacement'] = concentric_displacement
        rep['mean_power'] = mean_power
        rep['peak_power'] = peak_power
        rep['mean_propulsive_power'] = mean_propulsive_power
        rep['eccentric_duration'] = rep['bottom_time'] - rep['start_time']
        rep['concentric_duration'] = rep['concentric_peak_time'] - rep['bottom_time']
        rep['time_under_tension'] = rep['duration']
        
        # Aggiungi alla lista delle reps valide con metriche
        reps_with_metrics.append(rep)
        
        print(f"\n📊 REP #{rep['rep_num']}")
        print(f"   ⏱️  Sequenza: START {rep['start_time']:.2f}s → BOTTOM {rep['bottom_time']:.2f}s → PEAK {rep['concentric_peak_time']:.2f}s → END {rep['end_time']:.2f}s")
        print(f"   ⏱️  Durate: TUT={rep['time_under_tension']:.2f}s | Ecc={rep['eccentric_duration']:.2f}s | Conc={rep['concentric_duration']:.2f}s")
        print(f"   📈 Magnitudine: {rep['mag_start']:.3f}g → {rep['mag_bottom']:.3f}g → {rep['mag_peak']:.3f}g → {rep['mag_end']:.3f}g")
        print(f"   🚀 Mean Velocity: {rep['mean_velocity']:.3f} m/s | MPV: {rep['mean_propulsive_velocity']:.3f} m/s")
        print(f"   ⚡ Peak Velocity: {rep['peak_velocity']:.3f} m/s | Time to Peak: {rep['time_to_peak_velocity']:.3f}s")
        print(f"   💪 Mean Power: {rep['mean_power']:.2f} W | Peak Power: {rep['peak_power']:.2f} W | MPP: {rep['mean_propulsive_power']:.2f} W")
        print(f"   📏 ROM (Concentric): {rep['concentric_displacement']:.3f} m")
    
    print("=" * 90)
    
    # Sostituisci valid_reps con reps_with_metrics per le statistiche
    valid_reps = reps_with_metrics
    
    # Statistiche aggregate e Velocity Loss
    if len(valid_reps) > 1:
        mean_vels = [r['mean_velocity'] for r in valid_reps]
        peak_vels = [r['peak_velocity'] for r in valid_reps]
        mpvs = [r['mean_propulsive_velocity'] for r in valid_reps]
        mean_powers = [r['mean_power'] for r in valid_reps]
        peak_powers = [r['peak_power'] for r in valid_reps]
        roms = [r['concentric_displacement'] for r in valid_reps]
        tuts = [r['time_under_tension'] for r in valid_reps]
        
        # Velocity Loss (VL%) - perdita tra prima e ultima ripetizione
        velocity_loss_percent = ((mean_vels[0] - mean_vels[-1]) / mean_vels[0]) * 100 if mean_vels[0] > 0 else 0.0
        
        print(f"\n📈 STATISTICHE AGGREGATE ({len(valid_reps)} ripetizioni)")
        print(f"   Mean Velocity: {np.mean(mean_vels):.3f} m/s (±{np.std(mean_vels):.3f})")
        print(f"   Peak Velocity: {np.mean(peak_vels):.3f} m/s (±{np.std(peak_vels):.3f})")
        print(f"   Mean Propulsive Velocity: {np.mean(mpvs):.3f} m/s (±{np.std(mpvs):.3f})")
        print(f"   Mean Power: {np.mean(mean_powers):.2f} W (±{np.std(mean_powers):.2f})")
        print(f"   Peak Power: {np.mean(peak_powers):.2f} W (±{np.std(peak_powers):.2f})")
        print(f"   ROM (Concentric): {np.mean(roms):.3f} m (±{np.std(roms):.3f})")
        print(f"   Time Under Tension: {np.mean(tuts):.2f}s (±{np.std(tuts):.2f})")
        print(f"\n📉 VELOCITY LOSS: {velocity_loss_percent:.1f}% (Rep 1 → Rep {len(valid_reps)})")
        if velocity_loss_percent > 20:
            print(f"   ⚠️  Alta fatica rilevata (VL > 20%)")
        elif velocity_loss_percent > 10:
            print(f"   ⚡ Fatica moderata (VL 10-20%)")
        else:
            print(f"   ✅ Fatica minima (VL < 10%)")
        print("=" * 90)
else:
    print("⚠️ Nessuna ripetizione valida rilevata")

## 8. Visualizzazione Ripetizioni Rilevate con VBT

In [ ]:
if len(valid_reps) > 0:
    # Estrai Y acceleration smoothed per il plot
    y_acc = df['Y_smooth'].values
    
    fig, axes = plt.subplots(2, 1, figsize=(16, 10), sharex=True)
    
    # SUBPLOT 1: Magnitudine con marker delle fasi
    ax1 = axes[0]
    ax1.plot(timestamps, mag, 'lightgray', alpha=0.5, linewidth=1, label='Raw')
    ax1.plot(timestamps, mag, 'purple', linewidth=2.5, label='Smoothed', alpha=0.8)
    
    # Zone baseline (pattern matching)
    ax1.axhline(y=baseline_value, color='blue', linestyle='--', linewidth=2, label=f'Baseline ({baseline_value:.3f}g)', alpha=0.7)
    ax1.axhline(y=baseline_upper, color='red', linestyle=':', linewidth=1.5, label=f'Zona Upper (+{BASELINE_ZONE*100:.0f}%)', alpha=0.5)
    ax1.axhline(y=baseline_lower, color='green', linestyle=':', linewidth=1.5, label=f'Zona Lower (-{BASELINE_ZONE*100:.0f}%)', alpha=0.5)
    ax1.fill_between(timestamps, baseline_lower, baseline_upper, color='yellow', alpha=0.1, label='Zona Stabile')
    
    # Marker per ogni ripetizione
    colors_rep = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink']
    for i, rep in enumerate(valid_reps):
        color = colors_rep[i % len(colors_rep)]
        
        # Start, Bottom, End - add labels only for first rep to avoid legend clutter
        ax1.scatter(rep['start_time'], rep['mag_start'], s=150, marker='v', color=color, 
                   edgecolors='black', linewidths=2, zorder=10, 
                   label=f"Rep {rep['rep_num']} Start" if i == 0 else None)
        ax1.scatter(rep['bottom_time'], rep['mag_bottom'], s=200, marker='s', color='red', 
                   edgecolors='black', linewidths=2, zorder=10,
                   label='Bottom (inversione)' if i == 0 else None)
        ax1.scatter(rep['end_time'], rep['mag_end'], s=150, marker='^', color=color, 
                   edgecolors='black', linewidths=2, zorder=10,
                   label='End' if i == 0 else None)
        ax1.scatter(rep['concentric_peak_time'], rep['mag_peak'], s=200, marker='*', 
                   color='gold', edgecolors='black', linewidths=2, zorder=10,
                   label='Concentric Peak (>1g)' if i == 0 else None)
        
        # Zona refrattaria dopo ogni rep
        refractory_end = rep['end_time'] + REFRACTORY_PERIOD
        ax1.axvspan(rep['end_time'], min(refractory_end, timestamps[-1]), 
                   color='gray', alpha=0.15, zorder=1)
    
    ax1.set_ylabel('Magnitudine (g)', fontweight='bold', fontsize=12)
    ax1.set_title('Rilevamento Ripetizioni con Pattern Matching', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.legend(loc='upper right', fontsize=9, ncol=2)
    
    # SUBPLOT 2: Accelerazione Y con velocità integrata
    ax2 = axes[1]
    ax2.plot(timestamps, y_acc, 'green', linewidth=2, alpha=0.7, label='Y Acceleration (g)')
    ax2.axhline(y=1.0, color='gray', linestyle='--', linewidth=1, alpha=0.5, label='Gravità (1g)')
    ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5, alpha=0.5)
    
    # Plot velocità per ogni ripetizione
    for i, rep in enumerate(valid_reps):
        color = colors_rep[i % len(colors_rep)]
        
        # Ricostruisci velocità per il plot usando magnitudine
        concentric_slice = slice(rep['bottom_idx'], rep['concentric_peak_idx'] + 1)
        mag_concentric = mag[concentric_slice]
        time_concentric = timestamps[concentric_slice]
        mag_accel_net = mag_concentric - baseline_value
        
        velocity_plot = np.zeros(len(mag_accel_net))
        for j in range(1, len(velocity_plot)):
            dt_step = time_concentric[j] - time_concentric[j-1]
            velocity_plot[j] = velocity_plot[j-1] + mag_accel_net[j] * dt_step
        
        # Plot su asse secondario
        ax2_vel = ax2.twinx()
        ax2_vel.plot(time_concentric, velocity_plot, color=color, linewidth=2.5, 
                    linestyle='--', alpha=0.8, label=f"Rep {rep['rep_num']} Velocity")
        ax2_vel.set_ylabel('Velocity (m/s)', fontweight='bold', fontsize=12, color='navy')
        ax2_vel.tick_params(axis='y', labelcolor='navy')
        ax2_vel.axhline(y=0, color='navy', linestyle=':', linewidth=0.5, alpha=0.3)
        
        # Marker bottom (inversione) e picco concentrico
        ax2.scatter(rep['bottom_time'], y_acc[rep['bottom_idx']], s=200, marker='o', 
                   color=color, edgecolors='black', linewidths=2, zorder=10, 
                   label=f"Rep {rep['rep_num']} Bottom" if i == 0 else None)
        ax2.scatter(rep['concentric_peak_time'], y_acc[rep['concentric_peak_idx']], s=200, marker='*', 
                   color=color, edgecolors='black', linewidths=2, zorder=10,
                   label='Concentric Peak' if i == 0 else None)
        
        # Aggiungi testo con metriche VBT complete vicino al picco concentrico
        text_x = rep['concentric_peak_time'] + 0.1
        text_y = y_acc[rep['concentric_peak_idx']]
        metrics_text = (f"Rep {rep['rep_num']}\n"
                       f"MV: {rep['mean_velocity']:.3f} m/s\n"
                       f"PV: {rep['peak_velocity']:.3f} m/s\n"
                       f"MPV: {rep['mean_propulsive_velocity']:.3f} m/s\n"
                       f"ROM: {rep['concentric_displacement']:.3f} m\n"
                       f"Power: {rep['mean_power']:.0f}W")
        ax2.text(text_x, text_y, metrics_text,
                fontsize=8, color=color, fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.5', facecolor='white', edgecolor=color, alpha=0.9),
                verticalalignment='center')
    
    ax2.set_ylabel('Y Acceleration (g)', fontweight='bold', fontsize=12)
    ax2.set_xlabel('Tempo (s)', fontweight='bold', fontsize=12)
    ax2.grid(True, alpha=0.3)
    ax2.legend(loc='upper left', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Visualizzazione completata")
else:
    print("⚠️ Nessuna ripetizione da visualizzare")

## 10. Riepilogo e Conclusioni

Questo notebook ha analizzato i dati dell'accelerometro durante l'esecuzione di back squat identificando le **5 fasi** del movimento:

### Fasi Identificate:
1. 🟦 **Baseline Start** - Posizione iniziale stabile in piedi
2. 🔴 **Eccentrica** - Inizio della discesa (fase negativa)
3. 🟢 **Concentrica** - Inizio della spinta verso l'alto (punto più basso)
4. 🟠 **Arresto** - Decelerazione e fine del movimento verso l'alto
5. 🔵 **Baseline End** - Ritorno alla posizione stabile

### Marker sui Grafici:
- **Quadrati blu** (□) = Baseline
- **Triangoli rossi giù** (▼) = Inizio fase eccentrica
- **Triangoli verdi su** (▲) = Inizio fase concentrica
- **Diamanti arancioni** (◆) = Arresto
- **Quadrati ciano** (□) = Ritorno baseline

### Analisi Completata:
✅ Identificazione automatica delle transizioni di fase  
✅ Marker visualizzati all'**inizio** di ogni fase  
✅ Analisi temporale delle durate eccentriche/concentriche  
✅ Statistiche complete per ogni ripetizione  
✅ Visualizzazione 3D della traiettoria completa